In [1]:
input_path = "input"
output_path = "output"
crs = "EPSG:2154"

In [2]:
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import itertools
import geopandas as gpd
import os

/home/shoerl/anaconda3/envs/lead_lyon/lib/python3.8/site-packages/geopandas/_compat.py:106: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.1-CAPI-1.14.2). Conversions between both will be slow.
  warnings.warn(


In [3]:
assert os.path.exists(input_path)

# Network

In [4]:
import networkx as nx
import pyrosm 

In [5]:
osm = pyrosm.OSM("%s/filtered_area.osm.pbf" % input_path)
df_nodes, df_edges = osm.get_network(nodes = True, network_type = "driving")

df_nodes = df_nodes.to_crs(crs)
df_edges = df_edges.to_crs(crs)

graph = osm.to_graph(df_nodes, df_edges, graph_type="networkx")
df_nodes = df_nodes[df_nodes["id"].isin(graph.nodes)]

In [6]:
# Calculate distance matrix

nodes = list(graph.nodes)
df_distances = []

with tqdm(total = np.cumsum(np.arange(len(nodes)))[-1]) as progress:
    progress.update()
    
    for i in range(len(nodes)): 
        df_distances.append(dict(
            from_node = nodes[i], to_node = nodes[i],
            distance = 0.0
        ))
        
        for j in range(i + 1, len(nodes)):
            df_distances.append(dict(
                from_node = nodes[i], to_node = nodes[j],
                distance = nx.algorithms.shortest_path_length(graph, nodes[i], nodes[j], weight = "length")
            ))
            
            progress.update()

df_distances = pd.DataFrame.from_records(df_distances)

In [7]:
df_distances.to_csv(
    "%s/vrp_distances.csv" % output_path, sep = ";"
)

In [8]:
# Write outnetwork

df_edges[
    df_edges["u"].isin(df_nodes["id"]) & df_edges["v"].isin(df_nodes["id"])
].to_file("%s/vrp_network.gpkg" % output_path, driver = "GPKG")

df_nodes.to_file("%s/vrp_nodes.gpkg" % output_path, driver = "GPKG")

In [9]:
df_nodes_file = df_nodes.copy()
df_nodes_file["x"] = df_nodes.geometry.x
df_nodes_file["y"] = df_nodes.geometry.y
df_nodes_file["location_id"] = df_nodes_file["id"]
df_nodes_file[["location_id", "x", "y"]].to_csv("%s/vrp_nodes.csv" % output_path, sep = ";")